In [1]:
# import lib
import tensorflow as tf
import numpy as np
import sys, os,cv2
from sklearn.utils import shuffle
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle

In [2]:
# read all of the data
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels
    
train_images = read_all_images("../../DataSet/STL10/stl10_binary/train_X.bin") / 255.0
train_labels = read_labels    ("../../DataSet/STL10/stl10_binary/train_Y.bin")
test_images  = read_all_images("../../DataSet/STL10/stl10_binary/test_X.bin")  / 255.0
test_labels  = read_labels    ("../../DataSet/STL10/stl10_binary/test_y.bin")

label_encoder= OneHotEncoder(sparse=False,categories='auto')
train_labels = label_encoder.fit_transform(train_labels.reshape((-1,1)))
test_labels  = label_encoder.fit_transform(test_labels.reshape((-1,1)))

In [3]:
# some basic statistic of train and test image // hyper
print(train_images.max((0,1,2)),train_images.min((0,1,2)),train_images.mean((0,1,2)),train_images.std((0,1,2)) )
print(train_images.shape)
print(train_labels.shape)
print(test_images.max((0,1,2)),test_images.min((0,1,2)),test_images.mean((0,1,2)),test_images.std((0,1,2)) )
print(test_images.shape)
print(test_labels.shape)
num_epoch = 50 ; learning_rate = 0.0008; batch_size = 10

[1. 1. 1.] [0. 0. 0.] [0.44671062 0.43980984 0.40664645] [0.26034098 0.25657727 0.27126738]
(5000, 96, 96, 3)
(5000, 10)
[1. 1. 1.] [0. 0. 0.] [0.44723063 0.43964247 0.40495725] [0.2605645  0.25666146 0.26997382]
(8000, 96, 96, 3)
(8000, 10)


In [366]:
# import layers
def tf_softmax(x): return tf.nn.softmax(x)
def tf_elu(x):     return tf.nn.elu(x)
def d_tf_elu(x):   return tf.cast(tf.greater(x,0),tf.float32)  + (tf_elu(tf.cast(tf.less_equal(x,0),tf.float32) * x) + 1.0)
def tf_relu(x):    return tf.nn.relu(x)
def d_tf_relu(x):  return tf.cast(tf.greater(x,0),tf.float32)
def tf_iden(x): return x

class CNN():

    def __init__(self,k,inc,out, stddev=0.05,which_reg=0,act=tf_elu,d_act=d_tf_elu):
        self.w          = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float32))
        self.moving_w   = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float32))
        self.b          = tf.Variable(tf.zeros(out,dtype=tf.float32))
        self.m,self.v   = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.act,self.d_act = act,d_act
        self.which_reg  = which_reg
        
    def getw(self): return [self.w,self.b]
    def feedforward(self,input,stride=1,padding='VALID'):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding)
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient,stride=1,padding='VALID'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad_b      = tf.reduce_mean(grad_middle,(0,1,2))/batch_size
        grad        = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = tf.shape(self.w),  out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) / batch_size
        grad_pass   = tf.nn.conv2d_backprop_input (input_sizes = tf.shape(self.input),filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)

        if self.which_reg == 0:   grad = grad
        if self.which_reg == 0.5: grad = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)
        if self.which_reg == 1:   grad = grad + lamda * tf.sign(self.w)
        if self.which_reg == 1.5: grad = grad + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w
        if self.which_reg == 2:   grad = grad + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)
        if self.which_reg == 2.5: grad = grad + lamda * 2.0 * self.w
        if self.which_reg == 3:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)
        if self.which_reg == 4:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []
        
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middle = m_hat * learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle  )))
        
        update_w.append(tf.assign( self.mb,self.mb*beta1 + (1-beta1) * (grad_b)   ))
        update_w.append(tf.assign( self.vb,self.vb*beta2 + (1-beta2) * (grad_b ** 2)   ))
        m_hatb = self.mb / (1-beta1) ; v_hatb = self.vb / (1-beta2)
        adam_middleb = m_hatb * learning_rate/(tf.sqrt(v_hatb) + adam_e)
        update_w.append(tf.assign(self.b,tf.subtract(self.b,adam_middleb  )))
        
        return grad_pass,update_w
    
    def updatew(self):
        w,h,inc,outc = self.w.shape
        
        tempw = tf.reshape(self.w,(w*h,inc,outc))
        tempw = tf.transpose(tempw,(1,2,0))
        s,U,V = tf.svd(tempw)
        print(s)
        print(tf.reduce_min(s,1,True))
        scaleds = (s-tf.reduce_min(s,1,True))/(tf.reduce_max(s,1,True)-tf.reduce_min(s,1,True))
        _,n   = s.shape
        n     = n//2
        print(n)
        neww  = U[:,:,:n] @ tf.matrix_diag(scaleds)[:,:n,:] 
        neww  = tf.transpose(neww,(2,0,1))
        neww  = tf.reshape(neww,(w,h,inc,outc))
        
        update = []
        update.append(tf.assign(self.moving_w,self.moving_w * 0.9 + 0.1 * neww))
        update.append(tf.assign(self.w,self.moving_w))
        return update

In [367]:
# create the svd layer
class svd_layer():
    
    def __init__(self,batch,size,width):
        self.n = size
        self.moving_s = tf.Variable(tf.zeros((batch_size,size),dtype=tf.float32))
        self.moving_u = tf.Variable(tf.zeros((batch_size,width**2,size),dtype=tf.float32))
        self.moving_v = tf.Variable(tf.zeros((batch_size,size,size),dtype=tf.float32))
    
    def feedforward(self,data,training_phase):
        
        with tf.device('/cpu:0'):
            s,U,V = tf.svd(data)
        print(s)
        print(U)
        print(V)
        smin = tf.reduce_min(s,1,keepdims=True)
        smax = tf.reduce_max(s,1,keepdims=True)
        scaleds = (s - smin)/(smax-smin + 1e-8)
        def training_fn():
            data      = U[:,:,:] @ tf.matrix_diag(s)[:,:,:] @ tf.transpose(V,(0,2,1))[:,:,:]
            data = data  * tf.reduce_mean(tf.transpose(tf.abs(V),(0,2,1)) * scaleds[:,:,None] ,(1),keepdims=True)
            # data      = data * tf.reduce_mean(tf.transpose(tf.abs(V),(0,2,1)) * scaleds[:,:,None] ,(1),keepdims=True)
            update = []
            update.append(tf.assign(self.moving_u,self.moving_u*0.9 + 0.1 * U))
            update.append(tf.assign(self.moving_v,self.moving_v*0.9 + 0.1 * V))
            return data,update
            
        def testing_fn():
            data      = U[:,:,:] @ tf.matrix_diag(s)[:,:,:] @ tf.transpose(V,(0,2,1))[:,:,:]
            # data      = data * tf.reduce_mean(tf.transpose(tf.abs(V),(0,2,1)) * scaleds[:,:,None] ,(1),keepdims=True)
            update = []
            update.append(tf.assign(self.moving_u,self.moving_u))
            update.append(tf.assign(self.moving_v,self.moving_v))
            return data,update
        
        data,update  = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return data,update

In [368]:
# restart the graph 
# sess.close()
# tf.reset_default_graph()
learning_rate = 0.0008; batch_size = 100

l1 = CNN(3,3, 16)
l2 = CNN(3,16,16)
l3 = CNN(3,16,16)
l3_svd = svd_layer(100,16,24)

l4 = CNN(3,16,32)
l5 = CNN(3,32,32)
l6 = CNN(3,32,32)
l6_svd = svd_layer(100,32,6)

l7 = CNN(3,32,64)
l8 = CNN(3,64,64)
l9 = CNN(3,64,10)

In [369]:
# build graph 
x = tf.placeholder(shape=(batch_size,96,96,3),dtype=tf.float32)
y = tf.placeholder(shape=(batch_size,10),dtype=tf.float32)
is_train = tf.placeholder_with_default(False,())

layer1 = l1.feedforward(x       ,padding='SAME',stride=2)
layer2 = l2.feedforward(layer1  ,padding='SAME',stride=2) 
layer3 = l3.feedforward(layer2  ,padding='SAME',stride=1)

layer4 = l4.feedforward(layer3  ,padding='SAME',stride=2) 
layer5 = l5.feedforward(layer4  ,padding='SAME',stride=2)
layer6 = l6.feedforward(layer5  ,padding='SAME',stride=1)

layer7 = l7.feedforward(layer6  ,padding='SAME',stride=1)
layer8 = l8.feedforward(layer7  ,padding='SAME',stride=1) 
layer9 = l9.feedforward(layer8  ,padding='SAME',stride=1)

all_update = l1.updatew() + l2.updatew() + l3.updatew() + \
             l4.updatew() + l5.updatew() + l6.updatew() + \
             l7.updatew() + l8.updatew() + l9.updatew() 

final_layer = tf.reduce_mean(layer9,axis=(1,2))
cost        = tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_layer,labels=y)
auto_train  = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

final_softmax      = tf_softmax(final_layer)
correct_prediction = tf.equal(tf.argmax(final_softmax, 1), tf.argmax(y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Tensor("Svd_383:0", shape=(3, 9), dtype=float32)
Tensor("Min_391:0", shape=(3, 1), dtype=float32)
4
Tensor("Svd_384:0", shape=(16, 9), dtype=float32)
Tensor("Min_394:0", shape=(16, 1), dtype=float32)
4
Tensor("Svd_385:0", shape=(16, 9), dtype=float32)
Tensor("Min_397:0", shape=(16, 1), dtype=float32)
4
Tensor("Svd_386:0", shape=(16, 9), dtype=float32)
Tensor("Min_400:0", shape=(16, 1), dtype=float32)
4
Tensor("Svd_387:0", shape=(32, 9), dtype=float32)
Tensor("Min_403:0", shape=(32, 1), dtype=float32)
4
Tensor("Svd_388:0", shape=(32, 9), dtype=float32)
Tensor("Min_406:0", shape=(32, 1), dtype=float32)
4
Tensor("Svd_389:0", shape=(32, 9), dtype=float32)
Tensor("Min_409:0", shape=(32, 1), dtype=float32)
4
Tensor("Svd_390:0", shape=(64, 9), dtype=float32)
Tensor("Min_412:0", shape=(64, 1), dtype=float32)
4
Tensor("Svd_391:0", shape=(64, 9), dtype=float32)
Tensor("Min_415:0", shape=(64, 1), dtype=float32)
4


In [ ]:
# start the training 
# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())
num_epoch = 30000 ; avg_acc_train = 0; avg_acc_test  = 0; train_more = 1

for iter in range(num_epoch):
    
    test_images,test_labels   = shuffle(test_images,test_labels);

    for i in range(train_more):
        train_images,train_labels = shuffle(train_images,train_labels);
        for current_batch_index in range(0,len(train_images),batch_size):
            current_data  = train_images[current_batch_index:current_batch_index+batch_size].astype(np.float32)
            current_label = train_labels[current_batch_index:current_batch_index+batch_size].astype(np.float32)
            sess_results  = sess.run([accuracy,auto_train],feed_dict={x:current_data,y:current_label})
            sys.stdout.write(str(i) + ' Current Iter : ' + str(iter) + '/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(train_images)) + ' acc : ' + str(sess_results[0]) + '\r')
            sys.stdout.flush(); avg_acc_train = avg_acc_train + sess_results[0]
        
    for current_batch_index in range(0,len(test_images), batch_size):
        current_data  = test_images[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        current_label = test_labels[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        sess_results  = sess.run([accuracy],feed_dict={x:current_data,y:current_label})
        sys.stdout.write('Current Iter : ' + str(iter) + '/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(test_images)) + ' acc : ' + str(sess_results[0]) + '\r')
        sys.stdout.flush(); avg_acc_test = avg_acc_test + sess_results[0]        
        

    if avg_acc_train/(len(train_images)/batch_size)/train_more > 0.9 and avg_acc_test/(len(test_images)/batch_size) < 0.5 :  
        print('\n-----------reset')
        sess.run(all_update)
    if avg_acc_test/(len(test_images)/batch_size) > 0.5:
        print('-----------------------------------------')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print('-----------------------------------------')

    print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
    avg_acc_train = 0 ; avg_acc_test  = 0

Current : 0 Acc : 0.5501999992132187 Test Acc : 0.4532499972730875

Current : 1 Acc : 0.5513999980688095 Test Acc : 0.46212499812245367

Current : 2 Acc : 0.5718000024557114 Test Acc : 0.4528749980032444

Current : 3 Acc : 0.5822000032663346 Test Acc : 0.46037499867379666

Current : 4 Acc : 0.5914000022411346 Test Acc : 0.46399999782443047

Current : 5 Acc : 0.6077999985218048 Test Acc : 0.46312499791383743

Current : 6 Acc : 0.6226000010967254 Test Acc : 0.4641249947249889

Current : 7 Acc : 0.6413999998569488 Test Acc : 0.46674999594688416

Current : 8 Acc : 0.6484000027179718 Test Acc : 0.4563749972730875

Current : 9 Acc : 0.6692000043392181 Test Acc : 0.44937499761581423

Current : 10 Acc : 0.6896000027656555 Test Acc : 0.47649999558925626

Current : 11 Acc : 0.7181999969482422 Test Acc : 0.46312499307096006

Current : 12 Acc : 0.7177999997138977 Test Acc : 0.46712499521672723

Current : 13 Acc : 0.7619999992847443 Test Acc : 0.4692499987781048

Current : 14 Acc : 0.75619999885559

Current : 115 Acc : 0.6121999979019165 Test Acc : 0.42562499903142453

Current : 116 Acc : 0.6320000004768371 Test Acc : 0.41362499855458734

Current : 117 Acc : 0.6410000014305115 Test Acc : 0.4304999988526106

Current : 118 Acc : 0.663800002336502 Test Acc : 0.41474999748170377

Current : 119 Acc : 0.6835999989509582 Test Acc : 0.4454999964684248

Current : 120 Acc : 0.7332000029087067 Test Acc : 0.41862499825656413

Current : 121 Acc : 0.7505999994277954 Test Acc : 0.4366249989718199

Current : 122 Acc : 0.7551999998092651 Test Acc : 0.41849999837577345

Current : 123 Acc : 0.7943999969959259 Test Acc : 0.4379999995231628

Current : 124 Acc : 0.8500000023841858 Test Acc : 0.43212499767541884

Current : 125 Acc : 0.8378000009059906 Test Acc : 0.4102499973028898

Current : 126 Acc : 0.8470000004768372 Test Acc : 0.4127499971538782

Current : 127 Acc : 0.8855999982357026 Test Acc : 0.40949999913573265

Current Iter : 128/30000 batch : 7900/8000 acc : 0.3793
-----------reset
Current : 1

Current : 229 Acc : 0.8966000008583069 Test Acc : 0.40999999903142453

Current : 230 Acc : 0.885600004196167 Test Acc : 0.42949999757111074

Current Iter : 231/30000 batch : 7900/8000 acc : 0.4694
-----------reset
Current : 231 Acc : 0.9107999992370606 Test Acc : 0.42812499813735483

Current : 232 Acc : 0.10000000029802322 Test Acc : 0.10000000009313226

Current : 233 Acc : 0.10000000018626451 Test Acc : 0.10012499988079071

Current : 234 Acc : 0.10000000022351742 Test Acc : 0.10012499955482781

Current : 235 Acc : 0.1001999994367361 Test Acc : 0.10000000023283065

Current : 236 Acc : 0.11019999995827674 Test Acc : 0.102125000115484

Current : 237 Acc : 0.0997999995201826 Test Acc : 0.10000000027939678

Current : 238 Acc : 0.10000000029802322 Test Acc : 0.09999999986030161

Current : 239 Acc : 0.1 Test Acc : 0.100124999554827815

Current : 240 Acc : 0.10459999971091748 Test Acc : 0.14387500043958426

Current : 241 Acc : 0.16019999966025353 Test Acc : 0.16962499963119626

Current : 242 

Current : 342 Acc : 0.10000000029802322 Test Acc : 0.10012500016018748

Current : 343 Acc : 0.1303999999165535 Test Acc : 0.1706250005401671

Current : 344 Acc : 0.16100000068545342 Test Acc : 0.17724999990314244

Current : 345 Acc : 0.1844000007212162 Test Acc : 0.19637500001117586

Current : 346 Acc : 0.2029999989271164 Test Acc : 0.19662500014528633

Current : 347 Acc : 0.2183999991416931 Test Acc : 0.21500000078231096

Current : 348 Acc : 0.23599999994039536 Test Acc : 0.2508750006556511

Current : 349 Acc : 0.256199999153614 Test Acc : 0.2685000000521541

Current : 350 Acc : 0.2610000005364418 Test Acc : 0.24587499927729367

Current : 351 Acc : 0.2846000024676323 Test Acc : 0.2951250024139881

Current : 352 Acc : 0.3082000005245209 Test Acc : 0.30275000240653754

Current : 353 Acc : 0.3152000021934509 Test Acc : 0.32700000181794164

Current : 354 Acc : 0.34480000138282774 Test Acc : 0.3337500009685755

Current : 355 Acc : 0.3526000016927719 Test Acc : 0.3580000001937151

Current :

Current : 455 Acc : 0.29140000134706495 Test Acc : 0.3247500009834766

Current : 456 Acc : 0.321400004029274 Test Acc : 0.340875000320375

Current : 457 Acc : 0.337399999499321 Test Acc : 0.34825000185519456

Current : 458 Acc : 0.33940000116825103 Test Acc : 0.35075000040233134

Current : 459 Acc : 0.361599999666214 Test Acc : 0.3653749987483025

Current : 460 Acc : 0.38599999964237214 Test Acc : 0.3765000008046627

Current : 461 Acc : 0.382800001502037 Test Acc : 0.38812500052154064

Current : 462 Acc : 0.3946000009775162 Test Acc : 0.36612500064074993

Current : 463 Acc : 0.40900000035762785 Test Acc : 0.3996250007301569

Current : 464 Acc : 0.4145999979972839 Test Acc : 0.3972499992698431

Current : 465 Acc : 0.44179999887943266 Test Acc : 0.4274999961256981

Current : 466 Acc : 0.44999999821186065 Test Acc : 0.43099999949336054

Current : 467 Acc : 0.47459999442100526 Test Acc : 0.4306249964982271

Current : 468 Acc : 0.4817999958992004 Test Acc : 0.43312499634921553

Current : 46

...

Current : 2999 Acc : 0.5805999916791916 Test Acc : 0.47799999788403513

Current : 3000 Acc : 0.60299999833107 Test Acc : 0.49262499399483206

Current : 3001 Acc : 0.611199997663498 Test Acc : 0.4719999969005585

Current : 3002 Acc : 0.6159999960660935 Test Acc : 0.4933749962598085

Current : 3003 Acc : 0.6348000013828278 Test Acc : 0.4856249991804361

Current : 3004 Acc : 0.6458000016212463 Test Acc : 0.47449999749660493

Current : 3005 Acc : 0.6610000002384185 Test Acc : 0.483374996855855

Current : 3006 Acc : 0.6784000015258789 Test Acc : 0.48974999524652957

Current : 3007 Acc : 0.6944000041484832 Test Acc : 0.48162499703466893

Current : 3008 Acc : 0.7112000000476837 Test Acc : 0.48374999687075615

Current : 3009 Acc : 0.7284000015258789 Test Acc : 0.45837499797344206

Current : 3010 Acc : 0.7321999990940093 Test Acc : 0.47849999703466894

Current : 3011 Acc : 0.7464000070095063 Test Acc : 0.4849999964237213

Current : 3012 Acc : 0.7736000037193298 Test Acc : 0.4671249981969595

Cu

Current : 3111 Acc : 0.7206000006198883 Test Acc : 0.48637499548494817

Current : 3112 Acc : 0.7475999987125397 Test Acc : 0.4831249985843897

Current : 3113 Acc : 0.7593999981880188 Test Acc : 0.47949999794363973

Current : 3114 Acc : 0.7737999975681304 Test Acc : 0.47287499643862246

Current : 3115 Acc : 0.7971999990940094 Test Acc : 0.47474999725818634

Current : 3116 Acc : 0.8169999992847443 Test Acc : 0.47999999634921553

Current : 3117 Acc : 0.8448000025749206 Test Acc : 0.47199999652802943

Current : 3118 Acc : 0.8527999937534332 Test Acc : 0.4608749981969595

Current : 3119 Acc : 0.8637999987602234 Test Acc : 0.4529999945312738

Current : 3120 Acc : 0.8739999961853028 Test Acc : 0.4612499974668026

Current : 3121 Acc : 0.8986000001430512 Test Acc : 0.4662499960511923

Current Iter : 3122/30000 batch : 7900/8000 acc : 0.5496
-----------reset
Current : 3122 Acc : 0.9310000026226044 Test Acc : 0.47249999940395354

Current : 3123 Acc : 0.10419999990612268 Test Acc : 0.1001249996712

Current : 3223 Acc : 0.8942000019550324 Test Acc : 0.4709999989718199

Current Iter : 3224/30000 batch : 7900/8000 acc : 0.5596
-----------reset
Current : 3224 Acc : 0.9236000037193298 Test Acc : 0.4678749948740005

Current : 3225 Acc : 0.09739999990910292 Test Acc : 0.11162499964702874

Current : 3226 Acc : 0.1548000007867813 Test Acc : 0.17812500074505805

Current : 3227 Acc : 0.1703999997675419 Test Acc : 0.19412499964237212

Current : 3228 Acc : 0.1939999994635582 Test Acc : 0.2267499995417893

Current : 3229 Acc : 0.21780000045895576 Test Acc : 0.2523750001564622

Current : 3230 Acc : 0.2396000000834465 Test Acc : 0.23124999962747098

Current : 3231 Acc : 0.2527999997138977 Test Acc : 0.2737499998882413

Current : 3232 Acc : 0.27479999989271164 Test Acc : 0.2746250001713634

Current : 3233 Acc : 0.2871999993920326 Test Acc : 0.28012500125914813

Current : 3234 Acc : 0.3009999996423721 Test Acc : 0.30887500159442427

Current : 3235 Acc : 0.3184000024199486 Test Acc : 0.331125001423

Current : 3334 Acc : 0.2674000000953674 Test Acc : 0.29075000025331976

Current : 3335 Acc : 0.2858000019192696 Test Acc : 0.282875001616776

Current : 3336 Acc : 0.3020000013709068 Test Acc : 0.32075000274926424

Current : 3337 Acc : 0.32480000048875807 Test Acc : 0.34862500224262477

Current : 3338 Acc : 0.3364000016450882 Test Acc : 0.3403750015422702

Current : 3339 Acc : 0.3420000022649765 Test Acc : 0.35987500157207253

Current : 3340 Acc : 0.36280000269412993 Test Acc : 0.35512500144541265

Current : 3341 Acc : 0.37519999980926516 Test Acc : 0.3726249996572733

Current : 3342 Acc : 0.3979999989271164 Test Acc : 0.3899999987334013

Current : 3343 Acc : 0.41659999907016754 Test Acc : 0.4174999985843897

Current : 3344 Acc : 0.42379999816417696 Test Acc : 0.4010000001639128

Current : 3345 Acc : 0.43480000019073484 Test Acc : 0.4117499977350235

Current : 3346 Acc : 0.43439999759197234 Test Acc : 0.4343749977648258

Current : 3347 Acc : 0.4587999945878983 Test Acc : 0.4117499977350

Current : 3421 Acc : 0.7414000034332275 Test Acc : 0.49987499676644803

Current : 3422 Acc : 0.7603999984264374 Test Acc : 0.495374995470047

Current : 3423 Acc : 0.7848000013828278 Test Acc : 0.4923749968409538

Current : 3424 Acc : 0.7905999982357025 Test Acc : 0.4873749978840351

Current : 3425 Acc : 0.8095999991893769 Test Acc : 0.4932499974966049

Current : 3426 Acc : 0.8235999989509583 Test Acc : 0.49012499414384364

Current : 3427 Acc : 0.8437999975681305 Test Acc : 0.49349999725818633

Current : 3428 Acc : 0.8697999954223633 Test Acc : 0.49187499582767485

Current : 3429 Acc : 0.8927999997138977 Test Acc : 0.48424999639391897

Current : 3430 Acc : 0.8890000009536743 Test Acc : 0.4899999964982271

Current Iter : 3431/30000 batch : 7900/8000 acc : 0.5392
-----------reset
Current : 3431 Acc : 0.9110000038146973 Test Acc : 0.49037499651312827

Current : 3432 Acc : 0.09980000033974648 Test Acc : 0.12524999962188305

Current : 3433 Acc : 0.16019999995827675 Test Acc : 0.1728749998845

# Reference 
1. Brownlee, J. (2017). How to One Hot Encode Sequence Data in Python. Machine Learning Mastery. Retrieved 9 December 2018, from https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
2. tf.placeholder_with_default | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/placeholder_with_default
3. tf.nn.softmax_cross_entropy_with_logits | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits
4. line, O. (2018). Output without new line. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/2623470/output-without-new-line
5. shell?, H. (2018). How to tell if tensorflow is using gpu acceleration from inside python shell?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/38009682/how-to-tell-if-tensorflow-is-using-gpu-acceleration-from-inside-python-shell
6. GPU?, H. (2018). How to use TensorFlow GPU?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/51306862/how-to-use-tensorflow-gpu
7. Batch normalization: theory and how to use it with Tensorflow. (2018). Towards Data Science. Retrieved 9 December 2018, from https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
8. Batch normalization: theory and how to use it with Tensorflow. (2018). Towards Data Science. Retrieved 9 December 2018, from https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
9. tf.reset_default_graph | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/reset_default_graph
10. tf.Session | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/Session
11. tf.nn.moments | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/nn/moments
12. CMD?, H. (2018). How do I run two commands in one line in Windows CMD?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/8055371/how-do-i-run-two-commands-in-one-line-in-windows-cmd
13. loop, B. (2018). Batch script loop. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/2591758/batch-script-loop
14. tf.train.MomentumOptimizer | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/train/MomentumOptimizer
15. Test if two numpy arrays are (close to) equal, i. (2018). Test if two numpy arrays are (close to) equal, including shape. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/32874840/test-if-two-numpy-arrays-are-close-to-equal-including-shape
16. tf.linalg.diag | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/linalg/diag
17. tf.layers.batch_normalization | TensorFlow. (2018). TensorFlow. Retrieved 10 December 2018, from https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization
18. tf.layers.batch_normalization | TensorFlow. (2018). TensorFlow. Retrieved 10 December 2018, from https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization
19. error, t. (2018). tf.layers.batch_normalization large test error. Stack Overflow. Retrieved 10 December 2018, from https://stackoverflow.com/questions/43234667/tf-layers-batch-normalization-large-test-error
20. 
